### Population Tool Building Blocks


#### Block 1: Reading in data

Exploring ways that we can ingest the data from the source
Path to the Excel file: https://esa.un.org/unpd/wpp/DVD/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2017_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx

We want to pull the data and transform it into a format that will be useful for us to use.

In [ ]:
# Note: you can write comments in a block of code using ### first. Any text that follows a # will be 
# ignored by the python interpreter when you run that block of code.

# Import a library called pandas that has helpful functions that we'll need to download and process the data
import pandas as pd

# Assign the file path to a variable that we can call later
PATH = 'https://esa.un.org/unpd/wpp/DVD/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2017_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx'

In [ ]:
# We can show the contents of the PATH variable on screen. This is useful when troubleshooting!
print(PATH)

Now we'll try to pull the data from that excel file and load it into an object called a 'data frame'. To do this we'll use the read_excel function from the pandas library that we loaded above. You can read the official documentation about the read_excel function [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html).

In [ ]:
# Apply the read_excel function to the file path and store the results of 
# this into a new variable called 'first_data_test'

# Note that this takes a little longer to run. The block of code is actively running 
# while there is a * in the 'In [*]' box to the left; when it's done running tie * will be replaced by a number.

first_data_test = pd.read_excel(PATH)

What was read? Let's see by looking into the contents of first_data_test...

In [ ]:
# With pandas objects, you can display their contents simply by typing the name of the object.
first_data_test

In [ ]:
# You can also use print to show the contents, although it's not as pretty
print (first_data_test)

Because we're using pandas, what's printed out on the screen is the contents of an object called a pandas data frame. It has nice tidy columns and rows, just like an Excel spreadsheet.

What are the contents here? I opened up the same spreadsheet in Excel and compared what was shown above with what was in the spreadsheet. The sheets have a lot of information in the header, and the data tables don't start until further down. Based on what is in row 8, it looks like the information shown here is from the first tab of the excel file ('Estimates'). Luckily the read_excel function lets us also access the data in other sheets. Note that 'NaN' is what pandas puts into cells that are empty. 

Next let's try reading the second tab using the 'sheetname' argument to the read_excel function. We can also tell it to skip several rows at the beginning using the 'skiprows' argument.

*Note:* One funny thing about python is that all counting starts with 0 instead of 1. So what we think of as the 1st object in a sequence, python considers to be the 0th object. Therefore, in order to refer to the 2nd tab of the Excel (which contains the Medium Variant), we need to tell python to use the 1st sheet!

In [ ]:
# Now read the contents of the second sheet in the document, but skipping the first 14 rows
medium_variant = pd.read_excel(PATH, sheetname=1, skiprows=16)

# Output the results to see
medium_variant

#### Block 2: Exploring, selecting and filtering the data in pandas

In [ ]:
# Select and show the contents of the third column (column 2)
medium_variant[[2]]

In [ ]:
# Select and show the contents of the Region column and the 2017 column
medium_variant[['Region, subregion, country or area *','2017']]

In [ ]:
# Select the row(s) where the country equals Burundi
medium_variant.loc[medium_variant['Region, subregion, country or area *'] == 'Burundi']

My hunch is that the data would be easier to use if it were in 'long' format. That is, rather than having the years across the top as different columns, we pivot the data so that we have a separate row for each year and country combination. Pandas makes this transformation easy...

In [ ]:
# Pivot the data so that year is a new column, using the 'melt' method
medium_variant_long = pd.melt(medium_variant,
                              id_vars=['Index','Variant','Region, subregion, country or area *','Notes','Country code'],
                              value_name = 'Population',
                              var_name = 'Year')

medium_variant_long

Now we have a narrow but long data table that will be easier for us to use. It's gotten long quickly, though -- 23,478 rows!

In [ ]:
# Use the .loc method to select rows that meet our two criteria
medium_variant_long.loc[(medium_variant_long['Region, subregion, country or area *'] == 'Tonga') &
                   (medium_variant_long['Year'] == '2068')]

In [ ]:
# Just return the contents of the 'Population' column for the rows that meet our two criteria
medium_variant_long.loc[(medium_variant_long['Region, subregion, country or area *'] == 'Tonga') &
                   (medium_variant_long['Year'] == '2068'),'Population']

In [ ]:
# Return the value within the contents of that cell
medium_variant_long.loc[(medium_variant_long['Region, subregion, country or area *'] == 'Tonga') &
                   (medium_variant_long['Year'] == '2068'),'Population'].values[0]

I double checked this with row 288 and column BG of the original Excel file... and the number is the same!

#### Block 3: Asking for input

We need a way for our user to specify the year and the country they want. Eventually we can work on a pretty way to do this, but for now we can ask for input right in the python terminal using the input() function. The input they return gets stored in variables.

In [ ]:
country = input('Enter a country name:')
# I entered Tonga then hit enter

In [ ]:
# show the contents of the country variable
print (country)

In [ ]:
year = input('Enter a year:')
# I entered 2068 then hit enter

In [ ]:
# Use those inputted variables in our selection from Block 2:
medium_variant_long.loc[(medium_variant_long['Region, subregion, country or area *'] == country) &
                   (medium_variant_long['Year'] == year),'Population'].values[0]

In [ ]:
# Combine it together
country = input('Enter a country name:')
year = input('Enter a year:')

population = medium_variant_long.loc[(medium_variant_long['Region, subregion, country or area *'] == country) &
                   (medium_variant_long['Year'] == year),'Population'].values[0]

print('The population for', country, ' in the year ', year, ' is: ', population)

#### Next steps...
- We could combine all these blocks together into one script that pulls the data, reshapes it, asks a user to enter the country and year, then outputs the answer.
- What could go wrong? Great programs have anticipated every possible error or mistake that could happen, and have a fall-back plan. What happens if the link to the file changes? What happens if a user enters in an un-capitalised country, or mis-spells the country? What happens if the year they enter is outside the range of the projection? We could work on this using [if statements](http://www.afterhoursprogramming.com/tutorial/Python/If-Statement/) to check whether variables meet certain criteria.
- We pull other the gender and age-disaggregated tables and produce separate scripts that let a user interogate those datasets
- Eventually, we combine everything together and hopefully put a pretty front-end on it (although the front-end isn't my forte, so that will be a learning opportunity for me as well!)
- Anythin else?